In [9]:
import pandas as pd
import sqlite3
import os 

In [10]:
"""Connect to the sql database"""
database_path = "database.sqlite"
assert(os.path.exists(database_path)), "this notebook requires create_sql...py to have been run"
db = sqlite3.connect(database_path)
cur = db.cursor()


In [11]:
"""Read to dataframe"""
df = pd.read_sql("""SELECT * 
                FROM CRIMES
                WHERE "LSOA name" LIKE 'Barnet%'
                AND "Crime type" LIKE 'Burglary'""", con=db)
df

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,c2ccc34ed8f7e6c4529a2ee6464019d189970506889264...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.210051,51.657237,On or near Wentworth Road,E01000249,Barnet 001B,Burglary,Investigation complete; no suspect identified,None
1,bb5f9f0e9adf955886f1884e3eefc04cff57ac50f0b7da...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
2,0898a274117379acf52b279d2eaf103ede17ff9020a674...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
3,aaf394e5947f810a266a6e7dbc4b322206f92fbffe6aa5...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.203549,51.645780,On or near Supermarket,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
4,5feed401361e5c82e326e963773b22ae741369e2634b57...,2020-04,Metropolitan Police Service,Metropolitan Police Service,-0.205051,51.652412,On or near Cattley Close,E01000292,Barnet 001E,Burglary,Investigation complete; no suspect identified,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7452,fda0870be993d2c9352ae14a540bf529c15a7aa9d92a4e...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,None
7453,c6c2d9a772695865f4ca33fce59c26c48fe3a1cdac8c22...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,None
7454,e5bab2bd4f31de44515dbf125b85284392d0349139a1d3...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.197843,51.561093,On or near Church Walk,E01000140,Barnet 041C,Burglary,Under investigation,None
7455,823465427f8764e250a48312715a4bd87e8012412046c6...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.199449,51.563896,On or near Llanelly Road,E01000140,Barnet 041C,Burglary,Under investigation,None


In [12]:
import pandas as pd
import plotly.express as px

grouped = df.groupby('Month').size().reset_index(name='Count')

fig = px.line(grouped, x='Month', y='Count')
fig.update_layout(xaxis_title="Month", yaxis_title="Count")
fig.show()


In [13]:
import pandas as pd
import plotly.express as px
import calendar

df['Year'] = df['Month'].str[:4]
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df = df.sort_values('Month')
df['Month'] = df['Month'].dt.month
df['Month'] = df['Month'].apply(lambda x: calendar.month_name[x])
grouped = df.groupby(['Year', 'Month']).size().reset_index(name='Count')

# create a bar graph with separate boxes for each year
fig = px.bar(grouped, x='Month', y='Count', color='Year')
fig.update_layout(xaxis_title="Month", yaxis_title="Crime Count")

# show the plot
fig.show()


From this it is clear that most of the crimes are committed in october and november


In [14]:
fig = px.density_mapbox(df, lat='Latitude', lon='Longitude', radius=3, zoom = 11,
                        mapbox_style="stamen-terrain")
fig.show()

In [15]:
import pandas as pd
import plotly.express as px

# Create the density map
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', zoom=11, mapbox_style="stamen-terrain", color="Crime type")

mid_lat = (df['Latitude'].max() + df['Latitude'].min()) / 2
mid_lon = (df['Longitude'].max() + df['Longitude'].min()) / 2
hendon_lat = 51.599

top = df[(df['Latitude'] >= hendon_lat)]
bottom = df[(df['Latitude'] < hendon_lat)]

print(f'Above Hendon: {len(top)}')
print(f'Hendon and below: {len(bottom)}')

# Show the density map
fig.show()


Above Hendon: 4053
Hendon and below: 3404


In the future this should show other crimes too.

In [16]:
db.close()